In [ ]:
""" Jupyter notebook themes

Check the list of themes
!jt -l

My favourite dark mode is "onedork". Uncomment the first line below to set it, i.e.
!jt -t onedork -T

To revert to the default theme, uncomment the second line below, i.e.
!jt -r

For the change to take effect, just refresh the page. You don't need to restart the kernel
""";

# !jt -t onedork -T
# !jt -r

In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stat
from scipy.spatial import KDTree
import copy
import os

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import colormaps as mplcmaps
import matplotlib.cm as cm

from plotting.matplotlib_param_funcs import set_matplotlib_params,reset_rcParams
set_matplotlib_params()

In [ ]:
import src.variable_values_and_errors as val_err
from src.errorconfig import BootstrapConfig, MonteCarloConfig

import utils.miscellaneous_functions as MF
import utils.load_sim as load_sim
import utils.load_data as load_data
import utils.coordinates as coordinates

import plotting.map_functions as mapf
import plotting.mixed_plots as MP
import plotting.plotting_helpers as PH

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [ ]:
# degree_symbol = '°'
degree_symbol = "^\circ"

mass_density_label = r"$\Sigma \hspace{0.3} [\rm M_\odot kpc^{-2}]$"

In [ ]:
coolwarm = mplcmaps['coolwarm']
red = coolwarm(0.95)
blue = coolwarm(0.05)
green = 'darkgreen'
grey = 'lightgrey'

plt.rcParams['font.size'] = 20

In [ ]:
#CHOOSE

x_variable = "l"
y_variable = "b"

vel_x_variable = 'r'
vel_y_variable = 'l'

In [ ]:
symbol_dict = mapf.get_kinematic_symbols_dict(x_variable=x_variable,
                                                         y_variable=y_variable,
                                                         vel_x_variable=vel_x_variable,
                                                         vel_y_variable=vel_y_variable)

units_dict = mapf.get_kinematic_units_dict(degree_symbol=degree_symbol)

pos_symbols_dict,pos_units_dict = mapf.get_position_symbols_and_units_dict(degree_symbol=r"$%s$"%degree_symbol)

In [ ]:
symbol_dict["mean_b"] = r"$\langle |b| \rangle$"
units_dict["mean_b"] = r"$[^\circ]$"

In [ ]:
full_map_string_list,divergent_map_list = mapf.get_map_string_lists()

all_maps = False
full_map_string_list = [map_string for map_string in full_map_string_list if "spherical" not in map_string]

print(full_map_string_list)

# Path

In [ ]:
general_path = '/Users/luismi/Desktop/MRes_UCLan/'

In [ ]:
def get_base_path(single_variable):
    save_path = general_path+f'graphs/Observations/Apogee/individual_variable/{single_variable}/'
    MF.create_dir(save_path)
    
    return save_path

def get_save_path_spatial_cuts(single_variable, spatial_cuts_dict):
        
    save_path = get_base_path(single_variable)
    
    orders = {
        0: ["b","z"],
        1: ["d","R","x"],
        2: ["l","y"]
    }
    
    for o in orders:
        var = [v for v in spatial_cuts_dict if v in orders[o]]
        
        if len(var) == 0:
            continue
        elif len(var) > 1:
            raise ValueError(f"Did not expect more than one variable from `{orders[o]}`. If it was not a mistake, please specify the order.")
            
        variable = var[0]
            
        value_tuple = spatial_cuts_dict[variable]
        
        save_path += f"{MF.return_int_or_dec(value_tuple[0],2)}{variable}{MF.return_int_or_dec(value_tuple[1],2)}/"
        MF.create_dir(save_path)
    
    return save_path

def get_save_path_data(save_path_spatial, metal_lowcut, metal_lims):
        
    save_path = save_path_spatial + "data/"
    MF.create_dir(save_path)
    
    save_path += f"metal_lowcut_{metal_lowcut}/"
    MF.create_dir(save_path)
    
    save_path += f"{str(MF.return_int_or_dec(metal_lims[0],2))}metal{str(MF.return_int_or_dec(metal_lims[1],2))}/"
    MF.create_dir(save_path)
    
    return save_path

def get_save_path_sim(save_path_spatial, sim_choice, bar_angle, resampled_likedata_bool, age_lims, random_resampling_N, random_seed):
    
    save_path = save_path_spatial
    
    save_path += f"sim_{sim_choice}/" if sim_choice != "708main" else "sim/"
    MF.create_dir(save_path)
    
    save_path += f"bar_angle_{bar_angle}/"
    MF.create_dir(save_path)
    
    save_path += f"{MF.return_int_or_dec(age_lims[0],2)}age{MF.return_int_or_dec(age_lims[1],2)}/"
    MF.create_dir(save_path)
    
    save_path += "resampled_likedata/" if resampled_likedata_bool else ""
    MF.create_dir(save_path)
    
    save_path += f"random_resampling_{random_resampling_N}/" if random_resampling_N is not None else ""
    MF.create_dir(save_path)
    
    if random_resampling_N is not None:
        if random_seed is None:
            raise ValueError("You should set a random seed if performing random resampling for reproducibility reasons.")
        else:
            save_path += f"random_seed_{random_seed}/" if random_seed is not None else ""
            MF.create_dir(save_path)
        
    return save_path

def get_save_path_final(save_path_pop, binning_str, pop_str, error_type, montecarloconfig=None, bootstrapconfig=None):
    
    save_path = save_path_pop
    
    save_path += f"{binning_str}/"
    MF.create_dir(save_path)
    
    save_path += f"{pop_str}/"
    MF.create_dir(save_path)
    
    save_path_binning = save_path
    
    if error_type == "MC":
        if montecarloconfig is None:
            raise ValueError(f"montecarloconfig was None but it needs to be specified when working with MC errors.")
        
        save_path += f"MC_perturbed_{str.join(',',montecarloconfig.perturbed_vars)}/"
        MF.create_dir(save_path)
        
        if montecarloconfig.error_frac is not None:
            save_path += f"error_frac_{montecarloconfig.error_frac}/"
        else:
            if "data" not in save_path_pop:
                raise ValueError("error_frac was None and path does not correspond to observational data. Please specify how the MC errors are obtained.")
            save_path += f"data_uncertainties/"
        MF.create_dir(save_path)
        
        if montecarloconfig.repeats != 500:
            save_path += f"error_repeat_{montecarloconfig.repeats}/"
            MF.create_dir(save_path)
        
    elif error_type == "bootstrap":
        if bootstrapconfig is None:
            raise ValueError(f"bootstrapconfig was None but it needs to be specified when working with bootstrap errors.")
        
        save_path += "bootstrap"
        if bootstrapconfig.replace is False:
            save_path += "_noReplace"
        if bootstrapconfig.symmetric is False:
            save_path += "_Asymmetric"
        save_path += "/"
            
        MF.create_dir(save_path)
        
        if bootstrapconfig.sample_size is not None:
            save_path += f"bootsize_{bootstrapconfig.sample_size}/"
            MF.create_dir(save_path)
        
        if bootstrapconfig.repeats != 500:
            save_path += f"error_repeat_{bootstrapconfig.repeats}/"
            MF.create_dir(save_path)
    else:
        raise ValueError(f"error_type `{error_type}` not recognised.")

    return save_path, save_path_binning

# Load

In [ ]:
dummy_df = pd.DataFrame([[1,2,3],[2,3,1],[6,3,4]], columns=['a','b','c'])

In [ ]:
zabs = True
# zabs = False

R0 = 8.1

GSR = True
# GSR = False

## Sim

In [ ]:
sim_choice = "708main"
# sim_choice = "708mainDiff4"
# sim_choice = "708mainDiff5"

rot_angle = 27
axisymmetric = False
pos_scaling = 1.7

filename = load_sim.build_filename(choice=sim_choice,rot_angle=rot_angle,R0=R0,axisymmetric=axisymmetric,zabs=zabs,pos_factor=pos_scaling,GSR=GSR)

In [ ]:
np_path = general_path+f"data/{sim_choice}/numpy_arrays/"
        
df0 = load_sim.load_simulation(path=np_path,filename=filename)

In [ ]:
sim_resampled_likedata_bool = False

## Observations

In [ ]:
obs_errors = True
# obs_errors = False

data_zabs = True
# data_zabs = False

In [ ]:
data_path = general_path+"data/Observational_data/"

data = load_data.load_and_process_data(data_path=data_path, error_bool=obs_errors, zabs=zabs, R0=R0, GSR=GSR)

## Re-sample sim

In [ ]:
K = 10 # number of nearest neighbours to find

tree = KDTree(df0[["x","y","z"]].values)

already_chosen_indices = set()
nearest_simulation_star_indices = []
all_distances = []

for i, obs_star in enumerate(data[["x","y","z"]].values):
    k = K
    while True:
        distances, indices = tree.query(obs_star, k=k)# + len(already_chosen_indices))
        
        new_indices = [idx for idx in indices if idx not in already_chosen_indices]
        
        if len(new_indices) >= K:
            chosen_indices = new_indices[:K]
            
            nearest_simulation_star_indices.append(chosen_indices)            
            all_distances.append([dist for idx, dist in zip(indices, distances) if idx in chosen_indices])
            
            already_chosen_indices.update(chosen_indices)
            
            break
        else:
            k += K
            
#     clear_output(wait=True)
#     print(f"Iteration: {i+1}/{len(data)}")
            
nearest_simulation_star_indices = np.array(nearest_simulation_star_indices)
all_distances = np.array(all_distances)

In [ ]:
nearest_simulation_star_indices.shape, all_distances.shape

In [ ]:
save_path = f"{general_path}graphs/other_plots/resampled_sim/"

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot distance distribution

fig,ax=plt.subplots()

ax.hist(all_distances.flatten(),bins=300,log=False)
ax.set_xlabel("Nearest neighbour distances [kpc]")
ax.set_ylabel(r"$N$",rotation=0,labelpad=15)

max_distance = max(all_distances.flatten())
ax.axvline(x=max_distance,color="red",label="Maximum distance: %.2f [kpc]"%max_distance)
ax.legend()

if save_bool:
    plt.savefig(save_path + "distance_distribution",dpi=150,bbox_inches="tight")
    print("Saved:",save_path + "distance_distribution")
plt.show()

In [ ]:
if False: # lognormal fit to the distance distribution

    from scipy.stats import lognorm, norm, gaussian_kde

    # fit lognorm in linear scale
    shape, loc, scale = lognorm.fit(all_distances.flatten(), floc=0)
    mu, sigma = np.log(scale), shape

    plt.figure()
    count, bins, ignored = plt.hist(all_distances.flatten(), bins=200, density=True, alpha=0.3, color='b')
    pdf = lognorm.pdf(bins, sigma, scale=np.exp(mu))
    plt.plot(bins, pdf, color='r')
    plt.show()

    # fit gaussian in log scale
    log_distances = np.log(all_distances.flatten())
    mu, std = norm.fit(log_distances)

    plt.figure()
    count, bins, ignored = plt.hist(log_distances, bins=200, density=True, alpha=0.3, color='b')
    pdf = norm.pdf(bins, mu, std)
    plt.plot(bins, pdf, color='r')
    plt.show()

    # check KDE
    values = np.linspace(min(all_distances.flatten()), max(all_distances.flatten()), 1000)
    kde = gaussian_kde(all_distances.flatten())
    kde_values = kde(values)

    plt.figure()
    plt.hist(all_distances.flatten(), bins=200, density=True, alpha=0.3, color='b')
    plt.plot(values, kde_values, color='r')
    plt.show()

In [ ]:
sim_resampled = df0.iloc[nearest_simulation_star_indices.flatten()]

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
cuts_dict = {"R":[0,3.5],"age":[4,10]}#,"z":[0.5,2.2]}

cumulative = True
# cumulative = False

if True: # plot age distribution
    fig,ax=plt.subplots()

    bins = 500 if cumulative else 100

    ax.hist(MF.apply_cuts_to_df(df=df0, cuts_dict=cuts_dict)["age"],bins=bins,label="Original",density=True,cumulative=cumulative)
    ax.hist(MF.apply_cuts_to_df(df=sim_resampled, cuts_dict=cuts_dict)["age"],bins=bins,alpha=0.7,label=f"Resampled (k={K})",density=True,cumulative=cumulative)
    ax.legend()
    ax.set_xlabel("Age [Gyr]")
    ax.set_ylabel("Probability density" if not cumulative else "Fraction")

    filename = "age_distribution"
    filename += "_cumulative" if cumulative else ""

    for cut_variable in cuts_dict:
        if cut_variable != "age":
            l,r = cuts_dict[cut_variable]
            filename += f"_{l}{cut_variable}{r}"

    if save_bool:
        plt.savefig(save_path+filename+".png",dpi=150,bbox_inches="tight")
        print("Saved:",save_path+filename+".png")
    plt.show()

In [ ]:
save_bool = True
# save_bool = False

# density_bool = True
density_bool = False
bins_x = 100

if True: # xy and xz views
    fig,axs=plt.subplots(figsize=(9,11),nrows=2,gridspec_kw={"hspace":0})
    c1 = MP.quick_show_xy(sim_resampled,show=False,density=density_bool,bins_x=bins_x)
    c2 = MP.quick_show_xz(sim_resampled,show=False,zmin=0,density=density_bool,bins_x=bins_x)

    norm = PH.get_norm_from_count_list([c1,c2],log=True)

    MP.quick_show_xy(sim_resampled,ax=axs[0],norm=norm, density=density_bool,bins_x=bins_x)
    MP.quick_show_xz(sim_resampled,ax=axs[1],norm=norm,zmin=0, density=density_bool,bins_x=bins_x)

    cbar = plt.colorbar(cm.ScalarMappable(norm=norm,cmap="viridis"),ax=axs,shrink=0.8)
    cbar.set_label(mass_density_label) if density_bool else cbar.set_label(r"$N$",rotation=0,labelpad=20)

    for ax in axs:
        ax.set_aspect("equal")
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    if True: # filename and saving

        filename = f"resampled_sim_xyxz_k{k}"
        filename += f"_xbins{bins_x}"
        filename += "_N" if not density_bool else "_density"

        print(filename)

        if save_bool:
            print("Saving in:",save_path)

            plt.savefig(save_path+filename+".png", dpi=200,bbox_inches="tight")

        plt.show()

In [ ]:
sim_resampled_likedata_bool = True

if sim_resampled_likedata_bool:
    df0 = sim_resampled
    
    del sim_resampled

# Save arrays

## In bulk

In [ ]:
# data_bool = True
data_bool = False

In [ ]:
# get dataframe

if data_bool:
    # metal_lowcut = -9999
    metal_lowcut = -1

    data_trim = data[data['FeH']>=metal_lowcut]

    print(f"Chose minimum metallicity of {metal_lowcut}" if metal_lowcut != -9999 else "No minimum metallicity")

    if metal_lowcut != -9999:
        print(f"Removed {len(data)-len(data_trim)} ({MF.return_int_or_dec((len(data)-len(data_trim))/len(data)*100,2)}%) stars. {len(data_trim)} left")
        
    metallicity_median = MF.return_int_or_dec(np.median(data_trim["FeH"]),2)
    print("Median",metallicity_median,np.sum(data_trim['FeH']<metallicity_median),np.sum(data_trim['FeH']>metallicity_median))
    
    dataframe = data_trim
    
else:
    dataframe = df0

In [ ]:
def validate_dict(dic):
    if dic["error_type"] == "MC" and dic["montecarloconfig"] is None:
        raise ValueError("The montecarloconfig was None with MC error type.")
    elif dic["error_type"] != "MC" and dic["montecarloconfig"] is not None:
        raise ValueError(f"The montecarloconfig was set with `{dic['error_type']}` error type.")

    if dic["montecarloconfig"] is not None and list(dic["montecarloconfig"].affected_cuts_dict.keys())[0] not in dic["spatial_cuts"] | dic["pop_cut"]:
        raise ValueError("The affected cut in montecarloconfig is not in the spatial or pop dictionaries.")
        
    if dic["error_type"] == "bootstrap" and dic["bootstrapconfig"] is None:
        raise ValueError("The bootstrapconfig was None with bootstrap error type.")

In [ ]:
def get_minmax_ranges(df,x_var,x_min,x_max,binning_type,n_points):
    
    if binning_type == "equal_steps":
        x_edges = np.linspace(x_min,x_max,n_points+1)
        x_range_min = x_edges[:-1]
        x_range_max = x_edges[1:]
        
        pop_str = f"{n_points}_bins"
        
    elif binning_type == "equal_N":
        x_edges = PH.get_equal_n_bin_edges(df[x_var], n_points)

        x_range_max = x_edges[1:]
        x_range_min = x_edges[:-1]
        
        pop_str = f"{n_points}_bins"

    elif x_var == "b":
        
        if binning_type == "equal_number_low":
            
            if "FeH" not in df:
                raise ValueError("The equal_number_low latitude binning needs the data to be passed in the df argument.")
           
            low_max = np.max(df[df[x_var]<=6.8][x_var])
            n_points_low = n_points-2 if x_max == 13 else n_points-1
            edges_low = PH.get_equal_n_bin_edges(df[df[x_var]<=low_max][x_var], n_points_low)

            high_min = np.min(df[df[x_var]>6.8][x_var])
            high_max = np.max(df[df[x_var]<=9][x_var])

            x_range_min = list(edges_low[:-1]) + [high_min]
            x_range_max = list(edges_low[1:]) + [high_max]

            if x_max == 13:
                higher_min = np.min(df[df[x_var]>9][x_var])
                higher_max = np.max(df[x_var])

                x_range_min += [higher_min]
                x_range_max += [higher_max]

            x_range_min = np.array(x_range_min)
            x_range_max = np.array(x_range_max)
            
        elif binning_type == "custom_range":

            range_dict = {
                '1min': [0.5,  2.5,  4,    7],
                '1max': [2.5,  4,    6.1,  9],

                '1.5min': [0.5,2.5,4,7],
                '1.5max': [2.5,4,6.2,9],

                '2min': [1,2.5,4,7.1],
                '2max': [2.5,4,6.61,9]
            }

            if x_max==13:
                range_dict["2min"] += [10.4]
                range_dict["2max"] += [13]

            x_range_min = range_dict[str(x_max)+'min']
            x_range_max = range_dict[str(x_max)+'max']

            n_points = len(x_range_min)
        
        pop_str = f"{n_points}_bins"
    
    elif x_var in ["age","FeH"]:
        
        if binning_type == "0to9in1_oldSplit":
            pop_min_range = np.array([0,4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
            pop_max_range = np.array([4,5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])

        if binning_type == "4to9in1_oldSplit":
            pop_min_range = np.array([4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
            pop_max_range = np.array([5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])

        max_age_lim = 9
        limit_index = np.where(pop_max_range == max_age_lim)[0][0] + 1
        
        pop_str = binning_type
    
    try:
        return x_range_min,x_range_max,pop_str
    except:
        ValueError(f"Binning type `{binning_type}` not yet implemented, at least for x_var `{x_var}`.")

In [ ]:
if data_bool:

    all_save_dicts = [
        {
            "x_var": "FeH", 
            "x_min": -1, 
            "x_max": 0.61,
            "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"FeH":[-1,0.61]},
            "n_points": 4,
            "vel_freq": 1,
            "binning_type": "equal_N",
            "error_type": "MC",
            "montecarloconfig": MonteCarloConfig(perturbed_var="d", affected_cuts_dict={"R": [0,3.5]}, error_frac = 0.1)
        },
        {
            "x_var": "FeH", 
            "x_min": -1, 
            "x_max": 0.61,
            "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"FeH":[-1,0.61]},
            "n_points": 4,
            "vel_freq": 1,
            "binning_type": "equal_N",
            "error_type": "MC",
            "montecarloconfig": MonteCarloConfig(perturbed_var="d", affected_cuts_dict={"R": [0,3.5]}, error_frac = 0.2)
        },
        {
            "x_var": "FeH", 
            "x_min": -1, 
            "x_max": 0.61,
            "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"FeH":[-1,0.61]},
            "n_points": 4,
            "vel_freq": 1,
            "binning_type": "equal_N",
            "error_type": "bootstrap",
            "bootstrapconfig": BootstrapConfig()
        },
    ]

In [ ]:
if not data_bool:
    
    all_save_dicts = [
#         {
#             "x_var": "b", 
#             "x_min": 1.5, 
#             "x_max": 13,
#             "spatial_cut": {"b":[1.5,13],"d":[6.1,10.1],"l":[-2,2]},
#         },
#         {
#             "x_var": "age", 
#             "x_min": 4, 
#             "x_max": 10,
#             "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
#             "pop_cut": {"age":[4,10]},
#             "n_points": 8,
#             "vel_freq": 2,
#             "binning_type": "equal_steps",
#             "error_type": "MC",
#             "montecarloconfig": MonteCarloConfig(perturbed_var="d", affected_cuts_dict={"R": [0,3.5]}, error_frac = 0.1),
#             "random_resampling_N": 6000
#         },
#         {
#             "x_var": "age", 
#             "x_min": 4, 
#             "x_max": 10,
#             "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
#             "pop_cut": {"age":[4,10]},
#             "n_points": 8,
#             "vel_freq": 2,
#             "binning_type": "equal_steps",
#             "error_type": "MC",
#             "montecarloconfig": MonteCarloConfig(perturbed_var="d", affected_cuts_dict={"R": [0,3.5]}, error_frac = 0.2),
#             "random_resampling_N": 6000
#         },
#         {
#             "x_var": "age", 
#             "x_min": 4, 
#             "x_max": 10,
#             "spatial_cuts": {"b":[2,4],"R":[0,3.5],"l":[-2,2]},
#             "pop_cut": {"age":[4,10]},
#             "n_points": 20,
#             "vel_freq": 4,
#             "binning_type": "equal_steps",
#             "error_type": "bootstrap",
#             "bootstrapconfig": BootstrapConfig(),
#         },
        {
            "x_var": "age", 
            "x_min": 4, 
            "x_max": 10,
            "spatial_cuts": {"b":[3,6],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"age":[4,10]},
            "n_points": 10,
            "binning_type": "equal_steps",
            "error_type": "bootstrap",
            "bootstrapconfig": BootstrapConfig(repeats=500,replace=False,sample_size=950,symmetric=False),
        },
        {
            "x_var": "age", 
            "x_min": 4, 
            "x_max": 10,
            "spatial_cuts": {"b":[3,6],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"age":[4,10]},
            "n_points": 10,
            "binning_type": "equal_steps",
            "error_type": "bootstrap",
            "bootstrapconfig": BootstrapConfig(repeats=500,replace=False,sample_size=850,symmetric=False),
        },
        {
            "x_var": "age", 
            "x_min": 4, 
            "x_max": 10,
            "spatial_cuts": {"b":[3,6],"R":[0,3.5],"l":[-2,2]},
            "pop_cut": {"age":[4,10]},
            "n_points": 10,
            "binning_type": "equal_steps",
            "error_type": "bootstrap",
            "bootstrapconfig": BootstrapConfig(repeats=500,replace=False,sample_size=750,symmetric=False),
        },
    ]

In [ ]:
for d in all_save_dicts: # set defaults & validate
    if "random_resampling_N" not in d:
        d["random_resampling_N"] = None
    if "bootstrapconfig" not in d:
        d["bootstrapconfig"] = None
    if "montecarloconfig" not in d:
        d["montecarloconfig"] = None
    if "vel_freq" not in d:
        d["vel_freq"] = None

    validate_dict(d)

print("All dictionaries passed validation")

In [ ]:
min_star_number = 50 if data_bool else 50

velhist_bins = 50 if data_bool else 100

In [ ]:
random_seed = 0 # only has effect for dicts with random_resampling_N not None

In [ ]:
save_arrays_bool = True
# save_arrays_bool = False

In [ ]:
for dic in all_save_dicts:
    
    if True: # configure
        x_var,x_min,x_max = dic["x_var"],dic["x_min"],dic["x_max"]
        spatial_dict, pop_dict = dic["spatial_cuts"], dic["pop_cut"]
        n_points, vel_freq, binning_type = dic["n_points"],dic["vel_freq"],dic["binning_type"]
        random_resampling_N = dic["random_resampling_N"]
        error_type, montecarloconfig, bootstrapconfig = dic["error_type"], dic["montecarloconfig"],dic["bootstrapconfig"]

        all_cuts_dict = [spatial_dict, pop_dict]

        if True: # x_label
            if x_var in pos_symbols_dict:
                x_label = pos_symbols_dict[x_var] + f" [{pos_units_dict[x_var]}]"
            else:
                if x_var == "age":
                    x_label = "Age [Gyr]"
                elif x_var == "FeH":
                    x_label = "[Fe/H]"
                else:
                    raise ValueError("x_label not defined for the chosen variable")

        df_val = MF.apply_cuts_to_df(dataframe, cuts_dict=all_cuts_dict)

        if random_resampling_N is not None:
            np.random.seed(random_seed)
            df_val = df_val.iloc[np.random.choice(np.arange(len(df_val)), size=random_resampling_N, replace=False)]

        range_min,range_max,pop_str = get_minmax_ranges(df_val,x_var,x_min=x_min,x_max=x_max,binning_type=binning_type,n_points=n_points)

        # Saving the median because upon plotting we can always compute the mean but for the median we need the dataframe
        range_plot = PH.get_range_medians(df_val[x_var], range_min, range_max)

        print(f"Spatial cuts:",spatial_dict)
        if error_type == "MC":
            print(f"Post-MC cuts:",montecarloconfig.affected_cuts_dict)
        print(f"Pop cuts:",pop_dict)
        if random_resampling_N is not None:
            print(f"Random resampling to {random_resampling_N} stars")
        print(f"{len(df_val)} total stars")
        print(f"range_min: {range_min}")
        print(f"range_max: {range_max}")
        print(f"range_plot: {range_plot}")
        print("Star numbers:",stat.binned_statistic(values=None,x=df_val[x_var].values,bins=np.union1d(range_min,range_max),statistic="count")[0])

        save_path_spatial = get_save_path_spatial_cuts(single_variable=x_var,spatial_cuts_dict=spatial_dict)

        if data_bool:
            save_path_pop = get_save_path_data(save_path_spatial,metal_lims=pop_dict["FeH"],metal_lowcut=metal_lowcut)
        else:
            save_path_pop = get_save_path_sim(save_path_spatial,sim_choice=sim_choice,age_lims=pop_dict["age"],bar_angle=rot_angle,\
                                              resampled_likedata_bool=sim_resampled_likedata_bool,random_resampling_N=random_resampling_N,random_seed=random_seed)

        save_path,save_path_binning = get_save_path_final(save_path_pop, binning_str=binning_type,pop_str=pop_str, error_type=error_type,\
                                                          montecarloconfig=montecarloconfig, bootstrapconfig=bootstrapconfig)

        print("save_path:",save_path)
        
        if not os.path.isfile(save_path_binning+f"chosenRanges_{str(n_points*5)}bins.png"):
            MP.visualise_1D_binning(df_val[x_var],range_min,bin_edges_max=range_max,save_bool=True,save_path=save_path_binning,xlabel=x_label,log=False,\
                                   hist_bins=n_points*5)

        if not data_bool and not os.path.isfile(save_path_binning+f"chosenRanges_log_{str(n_points*5)}bins.png"):
            MP.visualise_1D_binning(df_val[x_var],range_min,bin_edges_max=range_max,save_bool=True,save_path=save_path_binning,xlabel=x_label,log=True,\
                                   hist_bins=n_points*5)

        if d["vel_freq"] is not None:
            save_path_hist = save_path_binning + "vel_histograms/"
            MF.create_dir(save_path_hist)

            save_path_hist += f"{velhist_bins}bins/"
            MF.create_dir(save_path_hist)

            print("Saving velocity histograms on\n",save_path_hist)

        df_err = None if error_type != "MC" else MF.apply_cuts_to_df(df=dataframe, cuts_dict=montecarloconfig.clean_value_cuts_dict(cuts_dict=all_cuts_dict))

    if True: # get arrays
        map_dict = {}

        for map_string in full_map_string_list:
            map_dict[map_string] = np.zeros(shape=(len(range_min)))

        for x_index, (xmin, xmax) in enumerate(zip(range_min,range_max)):

            print(xmin,xmax,end=";  ")

            include_lims = "both" if x_index==len(range_min)-1 else "min"
            x_lims_dict = {x_var:include_lims}
            x_cut_dict = {x_var:[xmin,xmax]}

            df_val_x = MF.apply_cuts_to_df(df_val, cuts_dict=x_cut_dict, lims_dict=x_lims_dict)

            if vel_freq is not None and x_index % vel_freq == 0:
                name_suffix = f"{str(MF.return_int_or_dec(xmin,dec=2))}{x_var}{str(MF.return_int_or_dec(xmax,dec=2))}"
                MP.plot_velocity_histograms_both_stats(df_val_x,vel_x_variable,vel_y_variable,save_bool=True,save_path=save_path_hist,suffix=name_suffix,verbose=False,bins=velhist_bins)
            
            if error_type == "MC":
                MC_config = copy.deepcopy(montecarloconfig)
                
                if random_resampling_N is not None:
                    MC_config.random_resampling_indices = df_val_x.index

                if x_var in MC_config.affected_cuts_dict:
                    MC_config.affected_cuts_dict[x_var] = x_cut_dict[x_var] # overwrite it using the more stringent cut
                    MC_config.affected_cuts_lims_dict[x_var] = x_lims_dict[x_var]

                    df_err_x = df_err # cut on x will be applied when performing the MC
                else:
                    df_err_x = MF.apply_cuts_to_df(df_err, cuts_dict=x_cut_dict, lims_dict=x_lims_dict)
            else:
                MC_config,df_err_x = None,None

            values = val_err.get_all_variable_values_and_errors(df_vals=df_val_x,df_errors=df_err_x,vel_x_var=vel_x_variable,vel_y_var=vel_y_variable,\
                                                                min_number = min_star_number, full_map_string_list=full_map_string_list,\
                                                                error_type=error_type, montecarloconfig=MC_config, bootstrapconfig=bootstrapconfig)

            if len(values) != len(full_map_string_list):
                raise ValueError("The length of the values list does not match the string list!")

            for map_string in full_map_string_list:
                map_dict[map_string][x_index] = values[map_string]

        print("Done computing map_dict")
        
    if True: # quick plot correlation
        x_plot = PH.get_range_means(minima=range_min,maxima=range_max) if not data_bool else PH.get_range_medians(df_val[x_var], range_min, range_max)

        fig,ax=plt.subplots(figsize=(5,5))
        ax.errorbar(x=x_plot, y=map_dict["correlation"],yerr=[map_dict["correlation_error_low"],map_dict["correlation_error_high"]],capsize=5)
        if not data_bool:
            ax.invert_xaxis()
        plt.show()

    if save_arrays_bool:

        array_path = save_path + "arrays/"

        overwrite = False
        if os.path.isdir(array_path):
            overwrite_str = input("There may be files already in this folder, do you want to overwrite them? Y/N\n")
            if overwrite_str.upper() == "Y":
                overwrite = True
        else:
            MF.create_dir(array_path)
            overwrite = True

        if overwrite:

            if True: # values as .txt and .npy

                with open(array_path+'values.txt','w') as f:
                    for key in map_dict:
                        f.write(key+'\n')
                        np.savetxt(f,map_dict[key],fmt='%.5f')
                        f.write('\n')

                for map_string in full_map_string_list:
                    np.save(array_path+map_string, map_dict[map_string])

            if True: # plot limits as .txt and .npy

                with open(array_path+'x_ranges.txt','w') as f:
                    f.write("x_range_min\n")
                    for mini in range_min:
                        f.write(f"{mini}\t")
                    f.write("\n\nx_range_max\n")
                    for maxi in range_max:
                        f.write(f"{maxi}\t")
                    f.write("\n\nx_range_plot\n")
                    for p in range_plot:
                        f.write(f"{p}\t")

                np.save(array_path+"x_range_min", range_min)
                np.save(array_path+"x_range_max", range_max)
                np.save(array_path+"x_range_plot", range_plot)

            print("Saved .txt and .npy in",array_path)
    else:
        print("Not saving")

    print("\n")

## Individually

### Spatial cuts

In [ ]:
x_var = "b"

x_min = 1
x_max = 13

if True: # x_label
    if x_var in pos_symbols_dict:
        x_label = pos_symbols_dict[x_var] + f" [{pos_units_dict[x_var]}]"
    else:
        if x_var == "age":
            x_label = "Age [Gyr]"
        elif x_var == "FeH":
            x_label = "[Fe/H]"
        else:
            raise ValueError("x_label not defined for the chosen variable")
    
MP.show_text(x_label)

In [ ]:
spatial_cuts_dict = {}

if x_var in pos_symbols_dict:
    spatial_cuts_dict[x_var] = [x_min,x_max]

spatial_cuts_dict["R"] = [0,2.5]
spatial_cuts_dict["l"] = [-2,2]

for variable in spatial_cuts_dict:
    print(variable, spatial_cuts_dict[variable])

In [ ]:
save_path_spatial = get_save_path_spatial_cuts(single_variable=x_var,cuts_dict=spatial_cuts_dict)

print(save_path_spatial)

### Sim

In [ ]:
data_bool = False

In [ ]:
# age_lims = [4,7]
age_lims = [9.5,10]

df = MF.apply_cuts_to_df(df=df0,cuts_dict=[spatial_cuts_dict,{"age":age_lims}])

#### Binning

In [ ]:
equal_steps = True

In [ ]:
if equal_steps:
    if x_var == "b":
        n_edges = 7 if x_max == 9 else 8
        n_points = n_edges - 1
    else:
        raise ValueError(f"Number of points not implemented for {x_var} yet.")

    x_edges = np.linspace(x_min,x_max,n_edges)

    x_range_min = x_edges[:-1]
    x_range_max = x_edges[1:]
    
    binning_str = "equal_steps"
    pop_str = f"{n_points}_bins"
    
# Saving the median because upon plotting we can always compute the mean but for the median we need the dataframe
x_range_plot = PH.get_range_medians(df[x_var], x_range_min, x_range_max)

print("Plotting at:",x_range_plot)
print("Number of datapoints:",n_points)

In [ ]:
fig,ax=plt.subplots(figsize=(10,2))

ax.errorbar(x=x_range_plot,xerr=PH.get_xerr(minima=x_range_min,maxima=x_range_max,plot=x_range_plot,frac=1),y=[0]*n_points,fmt="d",capsize=20)
ax.set_xlim(x_min-(x_max-x_min)/30,x_max+(x_max-x_min)/30)
ax.minorticks_on()
ax.set_xlabel(x_label)
ax.set_yticks([])
plt.show()

In [ ]:
# number of stars per bin
stat.binned_statistic(values=None,x=df[x_var].values,bins=x_edges,statistic="count")[0]

In [ ]:
save_path = get_save_path_sim(save_path_spatial, sim_choice=sim_choice, bar_angle=rot_angle, resampled_likedata_bool=sim_resampled_likedata_bool,\
                              age_lims=age_lims, random_resampling_N=random_resampling_N, random_seed=random_seed)

print(save_path)

### Data

In [ ]:
data_bool = True

In [ ]:
# metal_lowcut = -9999
metal_lowcut = -1

data_trim = data[data['FeH']>=metal_lowcut]

print(f"Chose minimum metallicity of {metal_lowcut}" if metal_lowcut != -9999 else "No minimum metallicity")

if metal_lowcut != -9999:
    print(f"Removed {len(data)-len(data_trim)} ({MF.return_int_or_dec((len(data)-len(data_trim))/len(data)*100,2)}%) stars. {len(data_trim)} left")

In [ ]:
metallicity_median = MF.return_int_or_dec(np.median(data_trim["FeH"]),2)
print(metallicity_median,np.sum(data_trim['FeH']<metallicity_median),np.sum(data_trim['FeH']>metallicity_median))

In [ ]:
metal_lims = [min(data_trim["FeH"]), metallicity_median]
# metal_lims = [metallicity_median, max(data_trim["FeH"])]

df = MF.apply_cuts_to_df(df=data_trim,cuts_dict=[spatial_cuts_dict,{"FeH":metal_lims}])

print(len(df))

#### Binning

In [ ]:
# plot

fig, ax = plt.subplots()

alpha=0.7
xmin = data_trim[x_var].min()
xmax = data_trim[x_var].max()
bins = np.linspace(xmin,xmax,50)

ax.hist(data_trim[x_var],bins=bins,label=fr'${str(MF.return_int_or_dec(min(data_trim["FeH"]),2))}\leq$[Fe/H]$\leq{str(MF.return_int_or_dec(max(data_trim["FeH"]),2))}$',\
        alpha=alpha,color='grey')

if len(data_trim) != len(df):
    ax.hist(df[x_var],bins=bins,label=fr'${str(MF.return_int_or_dec(metal_lims[0],2))}\leq$[Fe/H]$\leq{str(MF.return_int_or_dec(metal_lims[1],2))}$',\
            alpha=alpha,color='orange')

ax.set_xlim(xmin,xmax)
ax.axvline(x_min,color="orange");ax.axvline(x_max,color="orange")
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.set_xlabel(x_label)
ax.set_ylabel(r"$N$",rotation=0,labelpad=20)
ax.legend()

plt
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(20,5))
ax.hist(df[x_var],bins=500)
ticksss = np.arange(x_min,x_max,0.2)
ax.set_xticks(ticksss)
ax.set_xticklabels(labels=[str(np.float32(t)) for t in ticksss],size=8)
ax.set_xlabel(x_label)
plt.show()

In [ ]:
equal_number = False # divide in equal-number bins across all b
equal_number_low = True # divide in equal-number bins below |b|<6.61˚
custom_range = False
equal_steps = False # divide in constant latitude steps

plot_median_bool = True
# plot_median_bool = False # mid-point of bin

if x_var == "b":
    n_points = 3
    n_points += 1 if x_max == 13 else 0
else:
    raise ValueError(f"Number of points not implemented for {x_var} yet.")

print(f"{n_points} total points")

In [ ]:
assert equal_number+equal_number_low+custom_range+equal_steps == 1, "Choose a single range selection"

if x_var == "b":
    if equal_number:
        x_edges = PH.get_equal_n_bin_edges(df[x_var], n_points)

        x_range_max = x_edges[1:]
        x_range_min = x_edges[:-1]

        range_path = ""
    if equal_number_low:
        low_max = np.max(df[df[x_var]<6.8][x_var])
        n_points_low = n_points-2 if x_max == 13 else n_points-1
        edges_low = PH.get_equal_n_bin_edges(df[df[x_var]<=low_max][x_var], n_points_low)

    #     print(edges_low)

        high_min = np.min(df[df[x_var]>6.8][x_var])
        high_max = np.max(df[df[x_var]<9][x_var])

        x_range_min = list(edges_low[:-1]) + [high_min]
        x_range_max = list(edges_low[1:]) + [high_max]

    #     print(x_range_min,x_range_max)

        if x_max == 13:
            higher_min = np.min(df[df[x_var]>9][x_var])
            higher_max = np.max(df[x_var])

            x_range_min += [higher_min]
            x_range_max += [higher_max]

        x_range_min = np.array(x_range_min)
        x_range_max = np.array(x_range_max)
    elif custom_range:
        range_dict = {
            '1min': [0.5,  2.5,  4,    7],
            '1max': [2.5,  4,    6.1,  9],

            '1.5min': [0.5,2.5,4,7],
            '1.5max': [2.5,4,6.2,9],

            '2min': [1,2.5,4,7.1],
            '2max': [2.5,4,6.61,9]
        }

        if x_max==13:
            range_dict["2min"] += [10.4]
            range_dict["2max"] += [13]

        x_range_min = range_dict[str(lmax)+'min']
        x_range_max = range_dict[str(lmax)+'max']

        n_points = len(x_range_plot)    
    elif equal_steps:
        x_edges = np.linspace(x_min,x_max,n_points)
        x_range_min = x_edges[:-1]
        x_range_max = x_edges[1:]
else:
    raise ValueError(f"Binning not implemented for {x_var} yet.")

# Saving the median because upon plotting we can always compute the mean but for the median we need the dataframe
x_range_plot = PH.get_range_medians(df[x_var], x_range_min, x_range_max)

binning_str = np.array(["equal_number","equal_number_low","custom_range","equal_steps"])[np.array([equal_number,equal_number_low,custom_range,equal_steps])][0]
pop_str = f"{n_points}_bins"

print("Plotting at:",x_range_plot,"\n")
print(f"{n_points} datapoints")

In [ ]:
save_path = get_save_path_data(save_path_spatial, metal_lowcut, metal_lims, binning_str, pop_str)

print(save_path)

### Get arrays

In [ ]:
# save_bool = True
save_bool = False

if True: # visualise bulge cut
    
    fig,axs=plt.subplots(figsize=(15,10),nrows=2,sharex=True,gridspec_kw={"hspace":-0.43})
    
    MP.visualise_bulge_selection(given_axs=axs,cuts_dict=spatial_cuts_dict,y_max_plot=3)

#     MP.plot_circle(radius=2.5,ax=axs[0],linestyle="--",label=r"$R_\mathrm{GC}\leq2.5$ kpc")
#     axs[1].axvline(-2.5,color="k",linestyle="--")
#     axs[1].axvline(2.5,color="k",linestyle="--")

    if data_bool:
        axs[0].scatter(df["x"],df["y"],color="grey",s=1)
        axs[1].scatter(df["x"],df["z"],color="grey",s=1)
    else:
        MP.quick_show_xy(df=df,axs=axs[0])
        MP.quick_show_xz(df=df,axs=axs[1])
    
    if x_var in units_dict and units_dict[x_var] in ["$^\circ$","deg"]:
        for m,M in zip(x_range_min,x_range_max):
            MP.plot_angled_line(axs[1 if x_var == "b" else 0],xmin=-R0,ymin=0,xmax=4,angle=m,color="red",linestyle="--")
            MP.plot_angled_line(axs[1 if x_var == "b" else 0],xmin=-R0,ymin=0,xmax=4,angle=M,color="blue",linestyle="--")

    _ = [ax.legend(fontsize="x-small") for ax in axs]
    
#     fig.delaxes(axs[0])
    
    axs[1].set_xlim(-R0-0.1)

    filename = "illustrate_cuts"
    print(filename)
    if save_bool:
        print("Saving in:",save_path)
        plt.savefig(save_path+filename+".png",dpi=200,bbox_inches="tight")

In [ ]:
MP.visualise_1D_binning(df[x_var],x_range_min,bin_edges_max=x_range_max,save_bool=True,save_path=save_path,xlabel=x_label,log=False,\
                       hist_bins=400 if not data_bool else 100)

if not data_bool:
    MP.visualise_1D_binning(df[x_var],x_range_min,bin_edges_max=x_range_max,save_bool=True,save_path=save_path,xlabel=x_label,log=True,\
                           hist_bins=400)

In [ ]:
vel_hist_bool = True
# vel_hist_bool = False

velhist_bins = 100 if not data_bool else 50

if vel_hist_bool:
    save_path_hist = save_path + "vel_histograms/"
    MF.create_dir(save_path_hist)
    
    save_path_hist += f"{velhist_bins}bins/"
    MF.create_dir(save_path_hist)
    
    print("Saving velocity histograms on\n",save_path_hist)

In [ ]:
if True: # example vel hist
    MP.plot_velocity_histograms_both_stats(df[(df[x_var]>=min(x_range_min))&(df[x_var]<=min(x_range_max))],vel_x_variable,vel_y_variable,\
                                               bins=velhist_bins,colour_var="x",save_bool=False,suffix="example",verbose=True,show=True)

In [ ]:
bootstrap_repeat = 500

min_star_number = 100 if not data_bool else 50

In [ ]:
map_dict = {}
for map_string in full_map_string_list:
    map_dict[map_string] = np.zeros(shape=(len(x_range_min)))

for x_index, (xmin, xmax) in enumerate(zip(x_range_min,x_range_max)):

    print(xmin,xmax)
    
    include_lims = "both" if x_index==len(x_range_min)-1 else "min"
    df_x = MF.apply_cuts_to_df(df, cuts_dict={x_var:[xmin,xmax]}, lims_dict={x_var:include_lims})

    if vel_hist_bool:
        name_suffix = f"{str(MF.return_int_or_dec(xmin,dec=2))}{x_var}{str(MF.return_int_or_dec(xmax,dec=2))}"
        MP.plot_velocity_histograms_both_stats(df_x,vel_x_variable,vel_y_variable,save_bool=True,save_path=save_path_hist,suffix=name_suffix,verbose=x_index==0,bins=velhist_bins)

    values = val_err.get_all_variable_values_and_errors(df_x[f"v{vel_x_variable}"].values,df_x[f"v{vel_y_variable}"].values, full_map_string_list,\
                                                            repeat=bootstrap_repeat, min_number = min_star_number)   

    if len(values) != len(full_map_string_list):
        raise ValueError("The length of the values list does not match the string list!")

    for map_string in full_map_string_list:
        map_dict[map_string][x_index] = values[map_string]
    
del df,df_x
print("Done")

In [ ]:
map_dict["number"]

In [ ]:
# save arrays

array_path = save_path + "arrays/"
MF.create_dir(array_path)

if True: # values as .txt and .npy
            
    with open(array_path+'values.txt','w') as f:
        for key in map_dict:
            f.write(key+'\n')
            np.savetxt(f,map_dict[key],fmt='%.5f')
            f.write('\n')
    
    for map_string in full_map_string_list:
        np.save(array_path+map_string, map_dict[map_string])
        
if True: # plot limits as .txt and .npy

    with open(array_path+'x_ranges.txt','w') as f:
        f.write("x_range_min\n")
        for mini in x_range_min:
            f.write(f"{mini}\t")
        f.write("\n\nx_range_max\n")
        for maxi in x_range_max:
            f.write(f"{maxi}\t")
        f.write("\n\nx_range_plot\n")
        for p in x_range_plot:
            f.write(f"{p}\t")
    
    np.save(array_path+"x_range_min", x_range_min)
    np.save(array_path+"x_range_max", x_range_max)
    np.save(array_path+"x_range_plot", x_range_plot)
    
print("Saved .txt and .npy in",array_path)

# Plot

In [ ]:
def plot_number_bar(barax, plot_range, number_array,color,fill_bool=True,alpha=1,zorder=0, bar_width=50,bar_log=True,show_hist=False):
    if show_hist:
        # needs passing df
        barax.hist(df[x_var].values,bins=100,color="k",histtype="step")
    
    barax.bar(plot_range,number_array,width=bar_width,log=bar_log,color=color,edgecolor=color,alpha=alpha,zorder=zorder,fill=fill_bool,\
             linewidth=1.5 if not fill_bool else 0)
        
def plot_values_scatter_with_errors(ax, val_array, err_array, plot_range, min_range, max_range, color, label,line_alpha=1,zorder=0,\
                                    lines_bool=True,x_error_bool=True,marker=".",marker_size=5):
    xerror = PH.get_xerr(min_range,max_range,plot_range)

    ax.errorbar(plot_range,val_array,yerr=err_array,xerr=xerror if x_error_bool else None,capsize=capsize,color=color,label=label,\
                linestyle=None if lines_bool else '',alpha=line_alpha,zorder=zorder,marker=marker,markersize=marker_size)

def plot_values_surface(ax, val_array, err_array, plot_range,surface_color,line_color,label,surface_alpha=0.75,line_alpha=1,linestyle="-",zorder=0):
    
    if np.array(err_array).ndim == 2:
        ax.fill_between(plot_range,val_array-err_array[0],val_array+err_array[1],label=label,color=surface_color,alpha=surface_alpha,linewidth=0,zorder=zorder)
    else: # backwards compatibility
        ax.fill_between(plot_range,val_array-err_array,val_array+err_array,label=label,color=surface_color,alpha=surface_alpha,linewidth=0,zorder=zorder)
        
    ax.plot(plot_range,val_array,color=line_color,alpha=line_alpha,linestyle=linestyle,zorder=zorder)
        
def set_number_bar_axis_settings(barax,min_n,max_n,labels_on=True,min_shift_bool=True,max_shift_bool=True,hardcoded_lims=None):
    
    if hardcoded_lims is None:
        final_min = 10**MF.get_exponent(min_n) - (min_n/3 if min_shift_bool else 0)
        final_max = max_n + (10**MF.get_exponent(max_n) if max_shift_bool else 0)
    else:
        final_min,final_max = hardcoded_lims
    
    exponent_ticks = np.arange(MF.get_exponent(final_min),MF.get_exponent(final_max)+1,1)
    barax.set_yticks([10**i for i in exponent_ticks])
    
    barax.set_ylim(final_min,final_max)
            
    barax.yaxis.set_tick_params(which='minor', right=True,left=False)

    barax.tick_params(which='both',labelleft=False,labelright=labels_on)
    barax.tick_params(which='minor',labelright=False)

    if labels_on:
        barax.set_ylabel(r"$N$",rotation=0,labelpad=20)
        barax.yaxis.set_label_position("right")
    
def set_xaxis_settings(ax,xmin,xmax,xlabel,label_size="medium",xticks=None,xtick_labels=None, labels_on=True):
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    
    ax.set_xlim(xmin,xmax)
    
    if xticks is not None:
        ax.set_xticks(xticks, labels=xtick_labels)
    
    if labels_on:
        ax.set_xlabel(xlabel,fontsize=label_size)
    else:
        ax.set_xticklabels([])
        
def set_yaxis_settings(ax, map_string, label_size="medium",map_dict=None, labels_on=True,y_shift_dict={},symmetric_ylims_bool=False,\
                       set_ylims=True, hard_coded_ylims_bool=False, hard_coded_ylims_dict={}):
    
    if map_string == 'tilt_abs':
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))
    elif map_string == 'anisotropy':# or map_string == 'correlation':
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
        
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
#         ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    elif map_string == "correlation":
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.10))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(0.15))
    elif map_string in ["mean_vy"]:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
    elif map_string in ["std_vx","std_vy"]:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(25))
    
    if labels_on:
        ax.set_ylabel(symbol_dict[map_string]+units_dict[map_string],fontsize=label_size)
    else:
        ax.set_yticklabels([])
    
    def compute_ylims(map_dict, map_string):
        minimum = np.nanmin(map_dict[map_string]-map_dict[map_string+"_error_low"])
        maximum = np.nanmax(map_dict[map_string]+map_dict[map_string+"_error_high"])

        if map_string in yshift_dict:
            minimum -= yshift_dict[map_string]
            maximum += yshift_dict[map_string]

        if symmetric_ylims_bool:
            maxabs = np.nanmax(np.abs([minimum,maximum]))
            ax.set_ylim(-maxabs,maxabs)
        else:
            ax.set_ylim(minimum,maximum)
    
    if set_ylims:
        if hard_coded_ylims_bool and map_string in hard_coded_ylims_dict:
            ax.set_ylim(hard_coded_ylims_dict[map_string])
        else:
            if map_dict is None:
                raise ValueError("Cannot compute ylims if `map_dict` is None.")
            compute_ylims(map_dict, map_string)

def get_kinematic_label(map_string):
    return symbol_dict[map_string]+units_dict[map_string]

def get_legend_label(var_tuple,variable):
    if variable in pos_symbols_dict:
        var_symbol = pos_symbols_dict[variable]
        var_units = pos_units_dict[variable]
    else:
        if variable == "age":
            var_symbol = "Age"
            var_units = "Gyr"
        elif variable == "FeH":
            var_symbol = "[Fe/H]"
            var_units = ""
        else:
            raise ValueError("legend label not defined for the chosen variable")
    
    if variable == "R" and var_tuple[0] == 0:
        return var_symbol + fr"$\leq {var_tuple[1]}~$"+var_units
    else:
        return r"$%s\leq$"%str(var_tuple[0]) + var_symbol + (r"$/\mathrm{%s}$"%var_units if var_units!="" else "") + fr"$\leq{var_tuple[1]}$"

In [ ]:
def load_values_and_plot_ranges(path,full_map_string_list):
    
    map_dict = {}
    for m in full_map_string_list:
        map_dict[m] = np.load(f"{path}{m}.npy")
    
    min_range = np.load(path + f"x_range_min.npy")
    max_range = np.load(path + f"x_range_max.npy")
    plot_range = np.load(path + f"x_range_plot.npy")
    
    return map_dict, min_range, max_range, plot_range

## Prepare

In [ ]:
# Some reference values

l_cuts = [-2,2]
y_cuts = MF.return_int_or_dec_for_array([coordinates.ang_to_rect_1D(ang=l_cut,x=coordinates.get_solar_radius()) for l_cut in l_cuts])

R_variations = [[0,3.5],[0,2]]

print("l",l_cuts)
print("y",y_cuts)
print("R",R_variations)

try: # b and z based on data_trim
    b_range_min,b_range_max = PH.get_equal_n_minmax_b_ranges(data_trim)
    b_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(b_range_min,b_range_max)]

    z_range_min = [coordinates.ang_to_rect(ang=bmin,x=coordinates.get_solar_radius()) for bmin in b_range_min]
    z_range_max = [coordinates.ang_to_rect(ang=bmax,x=coordinates.get_solar_radius()) for bmax in b_range_max]
    z_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(z_range_min,z_range_max)]

    print("b",b_variations)
    print("z",z_variations)
except:
    pass

In [ ]:
line_alpha = 0.9
surface_alpha = 0.75
alpha_reduction_factor = 0.25

number_alpha = 0.75
number_alpha_reduction_factor = 0.5

##### Spatial variable

In [ ]:
pos_units_dict["b"] = r"$^\circ$"

In [ ]:
x_var = "b"

x_label = pos_symbols_dict[x_var] + f" [{pos_units_dict[x_var]}]"

MP.show_text(x_label)

In [ ]:
marker_size = 7.5

In [ ]:
cmap = mplcmaps["coolwarm"]

strong_colors = [
    cmap(0),
    cmap(cmap.N)
]

weak_colors = [
    cmap(int(cmap.N/3)),
    cmap(int(cmap.N*(1-1/3)))
]

for s,w in zip(strong_colors,weak_colors):
    MP.show_color(s)
    MP.show_color(w)

In [ ]:
# latitude 1b13; data & sim equalSteps; 1 column

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + "1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R2/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R2/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
        ],
        "title": "",
        "labels": [fr'[Fe/H]-rich ($-0.21$ to $0.61$)',fr'[Fe/H]-poor ($-1$ to $-0.21$)',"Young","Old"]+4*[None],
        "plot_ranges_str": 2*(2*["median"]+2*["mean"]),
        "surface_bools": 2*(2*[False]+2*[True]),
        "invert_xaxis": False,
        "xaxis_label": x_label,
        "x_ticks": np.arange(2,13+1,1),
        "x_lims": [1.5,13],
        "bar_widths": 8*[0.3]
    }   
}

save_path_plot = get_base_path(single_variable="b") + f"1.5b13/0R3.5_0R2/-2l2/both/data_equalN/data_n4_n4/sim_bar_angle_{bar_angle}/sim_equalSteps/sim_n8_n8/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5,4,3,2,1]
    all_dicts[key]["zorderNs"] = [8,7,4,3,5,6,2,1]
    all_dicts[key]["colors"] = 2*strong_colors+2*weak_colors
    
    all_dicts[key]["line_alphas"] = nplots*[line_alpha]
    all_dicts[key]["surface_alphas"] = nplots*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*(2*[False]+2*[True])
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
    all_dicts[key]["scatter_markers"] = 2*(["$\u25EF$","$\u25A1$"] + 2*[None])
    
surface_linestyle = "--"

In [ ]:
# latitude 1b13; data & sim equalN; 1 column

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + "1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_N/50_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_N/50_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R2/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + "1.5b13/0R2/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_N/50_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_N/50_bins/arrays/",
        ],
        "title": "",
        "labels": [fr'[Fe/H]-rich ($-0.21$ to $0.61$)',fr'[Fe/H]-poor ($-1$ to $-0.21$)',"Young","Old"]+4*[None],
        "plot_ranges_str": 2*(2*["median"]+2*["mean"]),
        "surface_bools": 2*(2*[False]+2*[True]),
        "invert_xaxis": False,
        "xaxis_label": x_label,
        "x_ticks": np.arange(2,13+1,1),
        "x_lims": [1.5,13],
        "bar_widths": 2*(2*[0.3]+2*[0.01])
    }   
}

save_path_plot = get_base_path(single_variable="b") + f"1.5b13/0R3.5_0R2.5/-2l2/both/data_equal_number_low/data_n4_n4/sim_bar_angle_{bar_angle}/sim_equalN/sim_n50_n50/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5,4,3,2,1]
    all_dicts[key]["zorderNs"] = [8,7,4,3,5,6,2,1]
    all_dicts[key]["colors"] = 2*strong_colors+2*weak_colors
    
    all_dicts[key]["line_alphas"] = nplots*[line_alpha]
    all_dicts[key]["surface_alphas"] = nplots*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*(2*[False]+2*[True])
    
    all_dicts[key]["number_bools"] = 4*[True] + 4*[False]
    
    all_dicts[key]["scatter_markers"] = 2*(["$\u25EF$","$\u25A1$"] + 2*[None])
    
surface_linestyle = "--"

In [ ]:
# latitude 1b13; sim -α and α bar angles; 1 column

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1b13/0R3.5/-2l2/sim/bar_angle_{-bar_angle}/4age7/equal_steps/7_bins/arrays/",
            get_base_path(single_variable="b") + f"1b13/0R3.5/-2l2/sim/bar_angle_{-bar_angle}/9.5age10/equal_steps/7_bins/arrays/",
            get_base_path(single_variable="b") + f"1b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/7_bins/arrays/",
            get_base_path(single_variable="b") + f"1b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/7_bins/arrays/"
        ],
        "title": "",
        "labels": [fr"Young ($\alpha={-bar_angle}$)",fr"Old ($\alpha={-bar_angle}$)",fr"Young ($\alpha={bar_angle}$)",fr"Young ($\alpha={bar_angle}$)"],
        "plot_ranges_str": 4*["mean"],
        "surface_bools": 4*[True],
        "invert_xaxis": False,
        "xaxis_label": x_label,
        "x_ticks": np.arange(1,13+1,1),
        "x_lims": [1,13],
        "bar_width": 0.3
    }   
}

save_path_plot = get_base_path(single_variable="b") + "1b13/0R3.5/-2l2/sim/bar_angles_-27,27/sim_equalSteps/sim_n7/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5]
    all_dicts[key]["zorderNs"] = [6,5,8,7]
    all_dicts[key]["colors"] = strong_colors+weak_colors
    
    all_dicts[key]["line_alphas"] = nplots*[line_alpha]
    all_dicts[key]["surface_alphas"] = nplots*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*[True]+2*[False]
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
surface_linestyle = "--"

In [ ]:
# latitude 1b13; 708mainDiff4 and 708mainDiff5 (with 708main as fiducial); 2 columns

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim_708mainDiff4/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim_708mainDiff4/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": "Weak bar",
        "labels": 4*[None],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim_708mainDiff5/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim_708mainDiff5/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": "Oval",
        "labels": ["Young","Old","Young (fiducial, strong bar)","Old (fiducial, strong bar)"]
    }
}

save_path_plot = get_base_path(single_variable="b") + "1.5b13/0R3.5/-2l2/sim_708mainDiff4__708mainDiff5/bar_angle_27/sim_equalSteps/sim_n8__n8/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5]
    all_dicts[key]["zorderNs"] = [6,5,8,7]
    all_dicts[key]["colors"] = strong_colors+weak_colors
    
    all_dicts[key]["line_alphas"] = 2*[line_alpha]+2*[0.8*line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[surface_alpha]+2*[0.8*surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*[True]+2*[False]
    
    all_dicts[key]["plot_ranges_str"] = 4*["mean"]
    all_dicts[key]["surface_bools"] = 4*[True]
    all_dicts[key]["invert_xaxis"] =  False
    all_dicts[key]["xaxis_label"] = x_label
    all_dicts[key]["x_ticks"] = np.arange(2,13+1,1)
    all_dicts[key]["x_lims"] = [1.5,13]
    all_dicts[key]["bar_widths"] = nplots*[0.3]
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
surface_linestyle = "-"

In [ ]:
# latitude 1b13; 708main Rgc and d cuts; symmetric & asymmetric; 4 columns

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": r"$R<2~$kpc",
        "labels": 4*[None],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<10.1$",
        "labels": ["Young","Old",r"Young ($R<3.5~$kpc)",r"Old ($R<3.5~$kpc)"]
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<9.1$",
        "labels": 4*[None]
    },
    3: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/"
        ],
        "title": r"$7.1<d/$kpc"+r"$<10.1$",
        "labels": 4*[None]
    }
}

save_path_plot = get_base_path(single_variable="b") + "1.5b13/0R2__6.1d10.1__6.1d9.1__7.1d10.1/-2l2/sim/bar_angle_27/4age7_9.5age10/sim_equalSteps/sim_n8/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5]
    all_dicts[key]["zorderNs"] = [6,5,8,7]
    all_dicts[key]["colors"] = strong_colors+weak_colors
    
    all_dicts[key]["line_alphas"] = 2*[line_alpha]+2*[0.8*line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[surface_alpha]+2*[0.8*surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*[True]+2*[False]
    
    all_dicts[key]["plot_ranges_str"] = 4*["mean"]
    all_dicts[key]["surface_bools"] = 4*[True]
    all_dicts[key]["invert_xaxis"] =  False
    all_dicts[key]["xaxis_label"] = x_label
    all_dicts[key]["x_ticks"] = np.arange(2,13+1,1)
    all_dicts[key]["x_lims"] = [1.5,13]
    all_dicts[key]["bar_widths"] = nplots*[0.3]
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
surface_linestyle = "-"

In [ ]:
# latitude 1b13; data Rgc and d cuts; symmetric & asymmetric; 4 columns

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$R<2~$kpc",
        "labels": 4*[None],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<10.1$",
        "labels": [fr'[Fe/H]-rich ($-0.21$ to $0.61$)',fr'[Fe/H]-poor ($-1$ to $-0.21$)',\
                   fr'[Fe/H]-rich ($-0.21$ to $0.61$, $R<3.5~$kpc)',fr'[Fe/H]-poor ($-1$ to $-0.21$, $R<3.5~$kpc)'],
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<9.1$",
        "labels": 4*[None]
    },
    3: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$7.1<d/$kpc"+r"$<10.1$",
        "labels": 4*[None]
    }
}

save_path_plot = get_base_path(single_variable="b") +\
                 "1.5b13/0R2__6.1d10.1__6.1d9.1__7.1d10.1/-2l2/data/metal_lowcut_-1/-1metal-0.21_-0.21metal0.61/data_equal_number_low/data_n4/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [8,7,6,5]
    all_dicts[key]["zorderNs"] = [6,5,8,7]
    all_dicts[key]["colors"] = strong_colors+weak_colors
    
    all_dicts[key]["line_alphas"] = 2*[line_alpha]+2*[0.8*line_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 2*[True]+2*[False]
    
    all_dicts[key]["plot_ranges_str"] = 4*["median"]
    all_dicts[key]["surface_bools"] = 4*[False]
    all_dicts[key]["invert_xaxis"] =  False
    all_dicts[key]["xaxis_label"] = x_label
    all_dicts[key]["x_ticks"] = np.arange(2,13+1,1)
    all_dicts[key]["x_lims"] = [1.5,13]
    all_dicts[key]["bar_widths"] = nplots*[0.3]
    
    all_dicts[key]["scatter_markers"] = 2*["$\u25EF$","$\u25A1$"]
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
surface_linestyle = "-"

In [ ]:
# latitude 1b13; data & 708main Rgc and d cuts; symmetric & asymmetric; 4 columns

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R2/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$R<2~$kpc",
        "labels": 8*[None],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d10.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<10.1$",
        "labels": ["Young","Old",r"Young ($R<3.5~$kpc)",r"Old ($R<3.5~$kpc)",\
                   fr'[Fe/H]-rich ($-0.21$ to $0.61$)',fr'[Fe/H]-poor ($-1$ to $-0.21$)',\
                   fr'[Fe/H]-rich ($-0.21$ to $0.61$, $R<3.5~$kpc)',fr'[Fe/H]-poor ($-1$ to $-0.21$, $R<3.5~$kpc)']
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/6.1d9.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$6.1<d/$kpc"+r"$<9.1$",
        "labels": 8*[None]
    },
    3: {
        "load_paths": [
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age7/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/sim/bar_angle_{bar_angle}/9.5age10/equal_steps/8_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/7.1d10.1/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-0.21metal0.61/equal_number_low/4_bins/arrays/",
            get_base_path(single_variable="b") + f"1.5b13/0R3.5/-2l2/data/metal_lowcut_-1/-1metal-0.21/equal_number_low/4_bins/arrays/"
        ],
        "title": r"$7.1<d/$kpc"+r"$<10.1$",
        "labels": 8*[None]
    }
}

save_path_plot = get_base_path(single_variable="b") +\
"1.5b13/0R2__6.1d10.1__6.1d9.1__7.1d10.1/-2l2/both/metal_lowcut_-1/-1metal-0.21_-0.21metal0.61/data_equal_number_low/data_n4/sim_bar_angle_27/4age7_9.5age10/sim_equalSteps/sim_n8/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [6,5,2,1,8,7,4,3]
    all_dicts[key]["zorderNs"] = [6,5,2,1,8,7,4,3]
    all_dicts[key]["colors"] = 2*(strong_colors+weak_colors)
    
    all_dicts[key]["line_alphas"] = 2*(2*[line_alpha]+2*[0.8*line_alpha])
    all_dicts[key]["surface_alphas"] = 2*(2*[surface_alpha]+2*[0.8*surface_alpha])
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    all_dicts[key]["number_filled"] = 4*[True]+4*[False]
    
    all_dicts[key]["plot_ranges_str"] = 4*["mean"]+4*["median"]
    all_dicts[key]["surface_bools"] = 4*[True]+4*[False]
    all_dicts[key]["invert_xaxis"] =  False
    all_dicts[key]["xaxis_label"] = x_label
    all_dicts[key]["x_ticks"] = np.arange(2,13+1,1)
    all_dicts[key]["x_lims"] = [1.5,13]
    all_dicts[key]["bar_widths"] = nplots*[0.3]
    
    all_dicts[key]["number_bools"] = nplots*[True]
    
    all_dicts[key]["scatter_markers"] = 4*["$\u25EF$","$\u25A1$"]
    
surface_linestyle = "--"

##### Kinpop

In [ ]:
pos_units_dict["b"] = "deg"

surface_linestyle = "-"

marker_size = 5

In [ ]:
# see https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3

# colors = ["blue","blue","blue"]
colors = ["#1b9e77","#d95f02","#7570b3"]

In [ ]:
# sim 10% and 20% distance errors, and bootstrap errors. 3 columns

bar_angle = 27

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="age") + f"3b6/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/20_bins/MC_perturbed_d/error_frac_0.1/arrays/",
        ],
        "title": "10% distance errors",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="age") + f"3b6/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/20_bins/MC_perturbed_d/error_frac_0.2/arrays/",
        ],
        "title": "20% distance errors",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="age") + f"3b6/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/20_bins/bootstrap/arrays/",
        ],
        "title": "Bootstrap errors",
        "surface_colors": ["grey"],
        "line_colors": ["k"],
        "number_colors": ["k"]
    },
}

save_path_plot = get_base_path(single_variable="age") + \
            f"3b6/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/20_bins/bootstrap__MCd0.1__MCd0.2/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [0]
    all_dicts[key]["line_alphas"] = [line_alpha]
    all_dicts[key]["surface_alphas"] = [surface_alpha]
    all_dicts[key]["number_alphas"] = [number_alpha]
    all_dicts[key]["zorderNs"] = [0]
    
    all_dicts[key]["plot_ranges_str"] = nplots*["mean"]
    all_dicts[key]["surface_bools"] = nplots*[True]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [True]
    all_dicts[key]["labels"] = [None]
    all_dicts[key]["bar_widths"] = nplots*[0.1]
    all_dicts[key]["number_filled"] = [True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(4,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# sim bootstrap errors without replacement, 350,250,150,50 resampling sizes. 4 columns

bar_angle = 27
b_range = "2b4"
n_bins = 20

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap/arrays/",
        ],
        "title": "Bootstrap",
        "number_colors": ["k"],
        "surface_colors": ["grey"],
        "line_colors": ["k"],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap_no_replacement/bootsize_350/arrays/",
        ],
        "title": "350-star samples",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap_no_replacement/bootsize_250/arrays/",
        ],
        "title": "250-star samples",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    3: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap_no_replacement/bootsize_150/arrays/",
        ],
        "title": "150-star samples",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    4: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap_no_replacement/bootsize_50/arrays/",
        ],
        "title": "50-star samples",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    }
}

save_path_plot = get_base_path(single_variable="age") + \
            f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/equal_steps/{n_bins}_bins/bootstrap_normal_and_no_replacement/bootsize_None__350__250__150__50/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [0]
    all_dicts[key]["line_alphas"] = [line_alpha]
    all_dicts[key]["surface_alphas"] = [surface_alpha]
    all_dicts[key]["number_alphas"] = [number_alpha]
    all_dicts[key]["zorderNs"] = [0]
    
    all_dicts[key]["plot_ranges_str"] = nplots*["mean"]
    all_dicts[key]["surface_bools"] = nplots*[True]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [True]
    all_dicts[key]["labels"] = [None]
    all_dicts[key]["bar_widths"] = nplots*[0.1]
    all_dicts[key]["number_filled"] = [True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(4,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# sim randomly resampled - 10% and 20% distance errors, and bootstrap errors. 3 columns

bar_angle = 27
random_resample_N = 6000
random_seed = 0
error_repeat = 2000
b_range = "3b6"
n_bins = 6

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/random_resampling_{random_resample_N}"+\
            f"/random_seed_{random_seed}/equal_steps/{n_bins}_bins/MC_perturbed_d/error_frac_0.1/error_repeat_{error_repeat}/arrays/",
        ],
        "title": "10% distance errors",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/random_resampling_{random_resample_N}"+\
            f"/random_seed_{random_seed}/equal_steps/{n_bins}_bins/MC_perturbed_d/error_frac_0.2/error_repeat_{error_repeat}/arrays/",
        ],
        "title": "20% distance errors",
        "number_colors": ["blue"],
        "surface_colors": ["cyan"],
        "line_colors": ["blue"],
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/random_resampling_{random_resample_N}"+\
            f"/random_seed_{random_seed}/equal_steps/{n_bins}_bins/bootstrap/error_repeat_{error_repeat}/arrays/",
        ],
        "title": "Bootstrap errors",
        "surface_colors": ["grey"],
        "line_colors": ["k"],
        "number_colors": ["k"]
    },
}

save_path_plot = get_base_path(single_variable="age") + f"{b_range}/0R3.5/-2l2/sim/bar_angle_{bar_angle}/4age10/random_resampling_{random_resample_N}/"+\
                f"random_seed_{random_seed}/equal_steps/{n_bins}_bins/MCd0.1__MCd0.2__bootstrap/error_repeat_{error_repeat}/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [0]
    all_dicts[key]["line_alphas"] = [line_alpha]
    all_dicts[key]["surface_alphas"] = [surface_alpha]
    all_dicts[key]["number_alphas"] = [number_alpha]
    all_dicts[key]["zorderNs"] = [0]
    
    all_dicts[key]["plot_ranges_str"] = nplots*["mean"]
    all_dicts[key]["surface_bools"] = nplots*[True]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [True]
    all_dicts[key]["labels"] = [None]
    all_dicts[key]["bar_widths"] = nplots*[0.1]
    all_dicts[key]["number_filled"] = [True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(4,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# data intrinsic, 10% and 20% distance errors, and bootstrap errors. 4 columns

x_ticks_FeH = np.arange(-0.9,0.6+0.3,0.3).astype(np.float16)
x_tick_labels_FeH = [str(MF.return_int_or_dec(i,2)).replace("-","–") if i!=0 else '0.0' for i in x_ticks_FeH]

all_dicts = {
    0: {
        "load_paths": [
            get_base_path(single_variable="FeH") + f"3b6/0R3.5/-2l2/data/metal_lowcut_-1/-1metal0.61/equal_N/4_bins/MC_perturbed_d/data_uncertainties/arrays/",
        ],
        "title": "Data distance errors (median 4%)",
        "number_colors": ["teal"],
        "line_colors": ["teal"],
        "x_tick_labels": x_tick_labels_FeH[:-1] + [""]
    },
    1: {
        "load_paths": [
            get_base_path(single_variable="FeH") + f"3b6/0R3.5/-2l2/data/metal_lowcut_-1/-1metal0.61/equal_N/4_bins/MC_perturbed_d/error_frac_0.1/arrays/",
        ],
        "title": "10% distance errors",
        "number_colors": ["blue"],
        "line_colors": ["blue"],
        "x_tick_labels": x_tick_labels_FeH[:-1] + [""]
    },
    2: {
        "load_paths": [
            get_base_path(single_variable="FeH") + f"3b6/0R3.5/-2l2/data/metal_lowcut_-1/-1metal0.61/equal_N/4_bins/MC_perturbed_d/error_frac_0.2/arrays/",
        ],
        "title": "20% distance errors",
        "number_colors": ["blue"],
        "line_colors": ["blue"],
        "x_tick_labels": x_tick_labels_FeH[:-1] + [""]
    },
    3: {
        "load_paths": [
            get_base_path(single_variable="FeH") + f"3b6/0R3.5/-2l2/data/metal_lowcut_-1/-1metal0.61/equal_N/4_bins/bootstrap/arrays/",
        ],
        "title": "Bootstrap errors",
        "line_colors": ["k"],
        "number_colors": ["k"],
        "x_tick_labels": x_tick_labels_FeH
    },
}

save_path_plot = get_base_path(single_variable="FeH") + \
            f"3b6/0R3.5/-2l2/data/metal_lowcut_-1/-1metal0.61/equal_N/4_bins/MCdData__MCd0.1__MCd0.2__bootstrap/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [0]
    all_dicts[key]["line_alphas"] = [line_alpha]
    all_dicts[key]["number_alphas"] = [number_alpha]
    all_dicts[key]["zorderNs"] = [0]
    
    all_dicts[key]["plot_ranges_str"] = nplots*["median"]
    all_dicts[key]["surface_bools"] = nplots*[False]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [True]
    all_dicts[key]["labels"] = [None]
    all_dicts[key]["bar_widths"] = nplots*[0.1]
    all_dicts[key]["number_filled"] = [True]
    all_dicts[key]["scatter_markers"] = ["x"]
    
    all_dicts[key]["invert_xaxis"] = False
    all_dicts[key]["xaxis_label"] = "FeH"
    all_dicts[key]["x_lims"] = [-1,0.61]
    all_dicts[key]["x_ticks"] = x_ticks_FeH

## Paths

In [ ]:
for key in all_dicts:
    print(key)
    for path in all_dicts[key]["load_paths"]:
        print(path)
        if not os.path.isdir(path):
            raise ValueError("Path did not exist!")

In [ ]:
os.makedirs(save_path_plot,exist_ok=True)

print(save_path_plot)

## Plot

In [ ]:
capsize = 5

scatter_join_bool = True
# scatter_join_bool = False

In [ ]:
zero_line_color = "grey"
zero_line_alpha = 1

In [ ]:
def get_legend_params(ncols, all_dicts,number_bool):
    
    if ncols != len(all_dicts):
        raise ValueError("Expected the `ncols` to match the length of `all_dicts`.")
    
    legend_cols = []
    for k in all_dicts.keys():
        if any(all_dicts[k]["labels"]):
            legend_cols.append(k)
    
    if ncols > 1 and len(legend_cols) == 1:
        ncols_leg = len(np.unique(all_dicts[legend_cols[0]]["colors"], axis=0)) 
    else:
        ncols_leg = 1
    
    if ncols > 1 and len(legend_cols) == 1:
        # Legend above the plot
        
        if ncols == 2:
            loc_x = -0.65
        elif ncols == 3:
            loc_x = -0.6 if len(legend_cols) == 1 else 0.16
        elif ncols == 1:
            loc_x = -0.25
        else:
            loc_x = 0

        legend_loc = [loc_x, 1.65]

        if not any(any(all_dicts[k]["number_bools"]) for k in all_dicts.keys()):
            legend_loc[1] -= 0.4

        if not any(all_dicts[k]["title"] for k in all_dicts.keys()):
            legend_loc[1] -= 0.15
    else:
        legend_loc = "lower left"
    
    return legend_cols, legend_loc, ncols_leg

# legend_bool = True
legend_bool = False

legend_row = 1
# legend_row = len(map_list)

legend_cols, legend_loc, ncols_leg = get_legend_params(ncols, all_dicts,number_bool)

# legend_loc = (0.1,1.3); ncols_leg=2
# legend_loc = (-0.4,1.4); ncols_leg=2
# legend_loc = (-0.4,1.5); ncols_leg=2
# legend_loc = (-0.4,1.65); ncols_leg=2
legend_loc = (-0.6,1.56); ncols_leg=4
# legend_loc = (-1.03,1.56); ncols_leg=4
# legend_loc = "upper center"; ncols_leg=2
# legend_cols = [1]

if legend_bool: # print and check there are actually any labels to be plotted
    
    any_legend_bool = False
    for key in all_dicts:
        any_legend_bool = any_legend_bool or any(all_dicts[key]["labels"])
        
    if not any_legend_bool:
        legend_bool = False
        print("No plot with label, setting legend_bool=False")
    else:
        print(f"Showing a legend in these plot columns: {legend_cols}\n placed at loc: {legend_loc}\n with {ncols_leg} cols.")

In [ ]:
plt.rcParams["font.size"] = 22
plt.rcParams["figure.titlesize"] = "medium"
axis_labels_size = "large"
# legend_fontsize = 14.3
legend_fontsize = 15.5

In [ ]:
figsize_x = 20

# figsize_y = 1.41*figsize_x # aspect ratio of A4
# figsize_y = 1.5*figsize_x
# figsize_y = 2*figsize_x
# figsize_y = 1.2*figsize_x
# figsize_y = 0.75*figsize_x
figsize_y = 0.9*figsize_x

figsize_x /= 2 if ncols == 1 else 1

# barax_height_ratio = 0.4
barax_height_ratio = 0.32
# barax_height_ratio = 0.25

In [ ]:
# number bar stuff

number_bool = True
# number_bool = False

number_variations_bool = True # This boolean is only used for the filename - ensure it matches with how you constructed all_dicts
# number_variations_bool = False

bar_log = True
# bar_log = False

number_min_shift_bool = True
# number_min_shift_bool = False

number_max_shift_bool = True
# number_max_shift_bool = False

# hardcoded_number_ylims = None
hardcoded_number_ylims = [10,5000]
# hardcoded_number_ylims = [80,3500]
# hardcoded_number_ylims = [1000,220000]

if number_bool: # check there are actually any to be plotted
    
    any_number_bool = False
    for key in all_dicts:
        any_number_bool = any_number_bool or any(all_dicts[key]["number_bools"])
        
    if not any_number_bool:
        number_bool = False
        print("No plot with number bool to draw, setting number_bool=False")

In [ ]:
# ylims

hard_coded_ylims_bool = True # currently needed to ensure the y axis is shared correctly when there is more than 1 column
# hard_coded_ylims_bool = False

# symmetric_ylims_bool = True
symmetric_ylims_bool = False

hard_coded_ylims = {
    "tilt_abs": [-45,3],
#     "tilt_abs": [-45,7],
#     "correlation": [-0.48,0.03]
}

yshift_dict = {
    "tilt_abs": 2,
    "anisotropy": 0.01,
    "correlation": 0.005,
    "mean_vx": 1,
    "mean_vy": 1,
    "std_vx": 2,
    "std_vy": 2
}

In [ ]:
# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy","correlation","tilt_abs"]; map_list_string = "all"
map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "meanstdani"
# map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
# map_list = ["mean_vx","mean_vy"]; map_list_string = "velmeans"

In [ ]:
filename_suffix = ""

In [ ]:
remove_ticklabels = {
#     0: "bottom"
}

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

fig,axs = plt.subplots(figsize=(figsize_x,figsize_y),nrows=len(map_list)+1,ncols=ncols, facecolor='w',\
                       gridspec_kw={'hspace':0,'wspace':0,'height_ratios':[barax_height_ratio]+len(map_list)*[1]})

if number_bool: # number and title
    bar_n_min,bar_n_max = [10**30],[0]

    for col in all_dicts:
        dic = all_dicts[col]
        
        barax = axs[0] if ncols==1 else axs[0,col]
        
        barax.set_title(dic["title"])

        for i in range(len(dic["load_paths"])):
            if dic["number_bools"][i]:
                map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
                
                if dic["plot_ranges_str"][i] == "mean":
                    plot_range = PH.get_range_means(min_range,max_range)

                plot_number_bar(barax, plot_range, map_dict["number"],color=dic["number_colors"][i],alpha=dic["number_alphas"][i],\
                                   zorder=dic["zorderNs"][i], bar_width=dic["bar_widths"][i], fill_bool=dic["number_filled"][i],bar_log=bar_log)

                bar_n_min = min(bar_n_min, np.min(map_dict["number"]))
                bar_n_max = max(bar_n_max, np.max(map_dict["number"]))
    
    for col in all_dicts:
        
        barax = axs[0] if ncols==1 else axs[0,col]
        
        set_number_bar_axis_settings(barax,min_n=bar_n_min,max_n=bar_n_max,labels_on=col==ncols-1,\
                                     min_shift_bool=number_min_shift_bool,max_shift_bool=number_max_shift_bool,hardcoded_lims=hardcoded_number_ylims)        
else:
    for col in all_dicts:
        barax = axs[0] if ncols==1 else axs[0,col]
        fig.delaxes(barax)
        
        first_ax = axs[1] if ncols==1 else axs[1,col]
        
        first_ax.set_title(all_dicts[col])

for row,map_string in enumerate(map_list): # plot
    
    ymin,ymax = [float("inf")],[float("-inf")]
    
    for col in all_dicts:
        
        ax = axs[row+1] if ncols==1 else axs[row+1,col]
        
        set_yaxis_settings(ax,map_string,label_size=axis_labels_size, set_ylims=False,labels_on=col==0)
        ax.axhline(y=0,linestyle='--',color=zero_line_color,alpha=zero_line_alpha,zorder=0)
        
        dic = all_dicts[col]
        for i in range(len(dic["load_paths"])):
            
            map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
            
            plot_range = plot_range if dic["plot_ranges_str"][i]=="median" else PH.get_range_means(min_range,max_range)
            label = dic["labels"][i] if row == legend_row-1 else None
            
            try:
                error_array = [map_dict[map_string+"_error_low"], map_dict[map_string+"_error_high"]]
            except: # backwards compatibility
                error_array = map_dict[map_string+"_error"]
            
            if dic["surface_bools"][i]:
                plot_values_surface(ax,map_dict[map_string],error_array,plot_range,surface_color=dic["surface_colors"][i],line_color=dic["line_colors"][i],\
                                    label=label, surface_alpha=dic["surface_alphas"][i],line_alpha=dic["line_alphas"][i],linestyle=surface_linestyle,\
                                    zorder=dic["zorders"][i])
            else:
                plot_values_scatter_with_errors(ax,map_dict[map_string],error_array,plot_range,min_range,max_range,color=dic["line_colors"][i],\
                                                label=label,lines_bool=scatter_join_bool,line_alpha=dic["line_alphas"][i],zorder=dic["zorders"][i],\
                                                marker=dic["scatter_markers"][i],marker_size=marker_size)
            
            try:
                ymin = min(ymin, np.nanmin(map_dict[map_string]-map_dict[map_string+"_error_low"]))
                ymax = max(ymax, np.nanmax(map_dict[map_string]+map_dict[map_string+"_error_high"]))
            except: # backwards compatibility
                print("Using symmetric errors")
                ymin = min(ymin, np.nanmin(map_dict[map_string]-map_dict[map_string+"_error"]))
                ymax = max(ymax, np.nanmax(map_dict[map_string]+map_dict[map_string+"_error"]))
    
    if map_string in yshift_dict:
        ymin -= yshift_dict[map_string]
        ymax += yshift_dict[map_string]
    
    if hard_coded_ylims_bool and map_string in hard_coded_ylims:
        ymin_hard,ymax_hard = hard_coded_ylims[map_string]
        
#         ymin = max(ymin,ymin_hard)
#         ymax = min(ymax,ymax_hard)
        
        ymin = ymin_hard
        ymax = ymax_hard
    
    for col in all_dicts:
        ax = axs[row+1] if ncols==1 else axs[row+1,col]
        
        ax.set_ylim(ymin,ymax)
        
        if row in remove_ticklabels and col == 0:
            yticks = ax.get_yticks()
            mapf.remove_ticklabel(ax=ax,ticks=yticks[(yticks>=ymin)&(yticks<=ymax)],which_axis="y",which_tick=remove_ticklabels[row])
        
if legend_bool: # legend
    for col in all_dicts:
        if col in legend_cols:
            
            ax = axs[legend_row] if ncols==1 else axs[legend_row,col]
            
            ax.legend(loc=legend_loc,fontsize=legend_fontsize,ncols=ncols_leg)#,loc="lower left")
    
if True: # x-axis
        
    for col in all_dicts:
        
        dic = all_dicts[col]
    
        for row in range(len(map_list)+1): # include number bars
            
            ax = axs[row] if ncols==1 else axs[row,col]
            
            labels_on = row==len(map_list)
            
            x_ticks = dic["x_ticks"]
            
            if "x_tick_labels" in dic:
                xtick_labels = dic["x_tick_labels"]
            else:
                xtick_labels = [str(MF.return_int_or_dec(i,2)).replace("-","–") if i!=0 else '0.0' for i in x_ticks]
            
                if labels_on and ncols > 1:

                    if col > 0:
                        left_tick_idx = 0 if not dic["invert_xaxis"] else -1
                        left_lim_idx = 0 if not dic["invert_xaxis"] else 1
                        left_tick_on_edge = dic["x_lims"][left_lim_idx] == dic["x_ticks"][left_tick_idx]

                        previous_right_tick_idx = -1 if not all_dicts[col-1]["invert_xaxis"] else 0
                        previous_right_lim_idx = 1 if not all_dicts[col-1]["invert_xaxis"] else 0
                        previous_right_tick_on_edge = all_dicts[col-1]["x_lims"][previous_right_lim_idx] == all_dicts[col-1]["x_ticks"][previous_right_tick_idx]

                        if left_tick_on_edge and previous_right_tick_on_edge:
                            xtick_labels[left_tick_idx] = ""

                    if col < ncols - 1:
                        right_tick_idx = -1 if not dic["invert_xaxis"] else 0
                        right_lim_idx = 1 if not dic["invert_xaxis"] else 0
                        right_tick_on_edge = dic["x_lims"][right_lim_idx] == dic["x_ticks"][right_tick_idx]

                        previous_left_tick_idx = 0 if not all_dicts[col+1]["invert_xaxis"] else -1
                        previous_left_lim_idx = 0 if not all_dicts[col+1]["invert_xaxis"] else 1
                        previous_left_tick_on_edge = all_dicts[col+1]["x_lims"][previous_left_lim_idx] == all_dicts[col+1]["x_ticks"][previous_left_tick_idx]

                        if right_tick_on_edge and previous_left_tick_on_edge:
                            xtick_labels[right_tick_idx] = ""
                    
            set_xaxis_settings(ax,xmin=dic["x_lims"][0],xmax=dic["x_lims"][1],xlabel=dic["xaxis_label"],xticks=x_ticks,xtick_labels=xtick_labels,\
                               labels_on=labels_on, label_size=axis_labels_size)

            if dic["invert_xaxis"]:
                ax.invert_xaxis()
    
fig.align_labels()
    
if True: # save
    
    filename = "kinpop" if x_var not in pos_symbols_dict else f"{x_var}_" + map_list_string
    
    filename += "_noN" if not number_bool else ""
    
    filename += "_noNvar" if number_bool and not number_variations_bool else ""
    
    filename += "_noLeg" if nplots > 1 and not legend_bool else ""
    
    any_scatter_bool = False
    for key in all_dicts:
        any_scatter_bool = any(~np.array(all_dicts[key]["surface_bools"]))
    if any_scatter_bool and not scatter_join_bool:
        filename += "_noLines"
    
    filename += filename_suffix
    
    print(filename)
    
    if save_bool:
        print("Saving in",save_path_plot)
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path_plot+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()